<a href="https://colab.research.google.com/github/lustraka/puzzles/blob/main/AoC2021/AoC_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Code Puzzles
[Advent of Code 2021](https://adventofcode.com/2021) | [reddit/adventofcode](https://www.reddit.com/r/adventofcode/)

In [1]:
import requests
import pandas as pd
import numpy as np
import re
path = 'https://raw.githubusercontent.com/lustraka/puzzles/main/AoC2021/data/'

## [Day 13](https://adventofcode.com/2021/day/13): Transparent Origami
### Part I
- **Unknown**: The number of dots visible after completing just the first fold instruction on the transparent paper.
- **Data**: Coordinates of dots and folding instructions
- **Condition**:
  - [0,0] represents the top-left coordinates
  - `x` increases to the right, `y` increases downward
  - `fold along y=...` instruction means folding the paper up
  - `fold along x=xxx` instruction means folding the paper left
- **Plan**:
  - Initialize the matrix
  - Write a function for folding
  - Count the dots after folding


In [2]:
example = """6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5"""

In [3]:
def parse(input):
  lines = input.split('\n')
  dots = []
  folds = []
  for line in lines:
    if ',' in line:
      dots.append(np.array(line.split(',')).astype(int))
    if '=' in line:
      fold = re.findall('[x,y]=\d+', line)[0].split('=')
      folds.append([fold[0],int(fold[1])])
  return np.array(dots), folds

dots, folds = parse(example)
print(dots[:4])
print(folds)

[[ 6 10]
 [ 0 14]
 [ 9 10]
 [ 0  3]]
[['y', 7], ['x', 5]]


In [4]:
r = requests.get(path+'AoC2021_13.txt')
dots, folds = parse(r.text)
print(dots[-2:])
print(folds[-2:])

[[954 841]
 [960 278]]
[['y', 13], ['y', 6]]


In [5]:
pap = np.zeros(dots.max(axis=0)+1).astype(int)
for dot in dots:
  pap[dot[0], dot[1]] = 1
pap

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

**Beware** of slicing vs. indexing. The expression `pap[6,10]` adresses one element [6,10] in the array, whereas the expression `pap[[6,10]]` addresses two rows [6, :] and [10, :]

In [6]:
print(dots[0])
print(pap[dots[0]])
print(np.stack((pap[6, :], pap[10, :])))

[309 320]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


In [7]:
# Fold the paper up - indices
y = 7
for i in range(y):
  print(i, 2*y-i)

0 14
1 13
2 12
3 11
4 10
5 9
6 8


In [8]:
pap_y7 = pap[:,:y].copy()
for i in range(y):
  pap_y7[:,i] += pap[:, 2*y-i]
pap_y7.T

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [9]:
pap_y7[pap_y7>0].shape

(12,)

In [10]:
# Fold the paper left
x = 5
pap_x5 = pap_y7[:x,:].copy()
for i in range(x):
  pap_x5[i,:] += pap_y7[2*x-i, :]
pap_x5.T

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [11]:
pap_x5[pap_x5>0].shape

(0,)

In [12]:
def solve_part1(input):
  dots, folds = parse(input)
  # Initialize the paper
  pap = np.zeros(dots.max(axis=0)+1).astype(int)
  for dot in dots:
    pap[dot[0], dot[1]] = 1
  print('Initial shape:', pap.shape)
  
  # Calculate the first fold
  ax, val = folds[0][0], folds[0][1]
  print(ax, val)
  # Fold the paper left
  if ax == 'x':
    for i in range(val):
      pap[i, :] += pap[2*val-i, :]
    # Adjust shape of the paper
    pap = pap[:val, :]
  else:
    for i in range(val):
      pap[:, i] += pap[:, 2*val-i]
    # Adjust shape of the paper
    pap = pap[:, :val]
  print('Folded shape:', pap.shape)

  return pap[pap>0].shape[0]

solve_part1(example)

Initial shape: (11, 15)
y 7
Folded shape: (11, 7)


17

In [13]:
solve_part1(r.text)

Initial shape: (1311, 890)
x 655
Folded shape: (655, 890)


669

### Part II
- **Unknown**: Eight capital letters after folding the paper.
- **Data**: Same as in part I
- **Condition**:
  - Same as in part I
- **Plan**:
  - Calculate all folds and return transposed result


In [24]:
def solve_part2(input):
  dots, folds = parse(input)
  # Initialize the paper
  #pap = np.zeros(dots.max(axis=0)+1).astype(int)
  # Solve IndexError: index 894 is out of bounds for axis 1 with size 890
  pap = np.zeros((1311, 895)).astype(int)
  for dot in dots:
    pap[dot[0], dot[1]] = 1
  print('Initial shape:', pap.shape)
  
  # Calculate the folds
  for ax, val in folds:
    #print(ax, val)
    # Fold the paper left
    if ax == 'x':
      for i in range(val):
        pap[i, :] += pap[2*val-i, :]
      # Adjust shape of the paper
      pap = pap[:val, :]
    else:
      for i in range(val):
        pap[:, i] += pap[:, 2*val-i]
      # Adjust shape of the paper
      pap = pap[:, :val]
    #print('Folded shape:', pap.shape)

  return pap.T

code = solve_part2(example)
print(code)

Initial shape: (1311, 895)
[[1 1 1 1 1]
 [1 0 0 0 1]
 [1 0 0 0 1]
 [1 0 0 0 1]
 [1 2 2 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [25]:
code = solve_part2(r.text)
print(code)

Initial shape: (1311, 895)
[[10  0  0  7  0  2 13 37  7  0  1  1  1  7  0 26  2 20 31  0  0  1 16  0
   0  2  0  0 13  0  0  6 12  0  0  0  0  2  5  0]
 [12  0  0  5  0 10  0  0  0  0  8  0  0  0  0  0  0  0  1  0  8  0  0  1
   0  5  0  0  9  0 13  0  0  4  0  0  0  0 11  0]
 [ 4  0  0 16  0 24  5  8  0  0 21 17  3  0  0  0  0  9  0  0  3  0  0  0
   0  3  0  0 17  0  2  0  0  0  0  0  0  0  2  0]
 [ 1  0  0 14  0 12  0  0  0  0 19  0  0  0  0  0  7  0  0  0  3  0  0  0
   0  9  0  0  3  0 21  0  0  0  0  0  0  0  1  0]
 [ 5  0  0 15  0  8  0  0  0  0  3  0  0  0  0  2  0  0  0  0 27  0  0  4
   0 18  0  0  4  0  1  0  0 17  0  8  0  0 17  0]
 [ 0 20  2  0  0  1 10 13 15  0 10  0  0  0  0  6  6  3  2  0  0  1  4  0
   0  0  5 19  0  0  0  4  4  0  0  0  8  7  0  0]]


In [37]:
for j in range(5,41,5):
  print(code[:,j-5:j-1])

[[10  0  0  7]
 [12  0  0  5]
 [ 4  0  0 16]
 [ 1  0  0 14]
 [ 5  0  0 15]
 [ 0 20  2  0]]
[[ 2 13 37  7]
 [10  0  0  0]
 [24  5  8  0]
 [12  0  0  0]
 [ 8  0  0  0]
 [ 1 10 13 15]]
[[ 1  1  1  7]
 [ 8  0  0  0]
 [21 17  3  0]
 [19  0  0  0]
 [ 3  0  0  0]
 [10  0  0  0]]
[[26  2 20 31]
 [ 0  0  0  1]
 [ 0  0  9  0]
 [ 0  7  0  0]
 [ 2  0  0  0]
 [ 6  6  3  2]]
[[ 0  1 16  0]
 [ 8  0  0  1]
 [ 3  0  0  0]
 [ 3  0  0  0]
 [27  0  0  4]
 [ 0  1  4  0]]
[[ 2  0  0 13]
 [ 5  0  0  9]
 [ 3  0  0 17]
 [ 9  0  0  3]
 [18  0  0  4]
 [ 0  5 19  0]]
[[ 0  6 12  0]
 [13  0  0  4]
 [ 2  0  0  0]
 [21  0  0  0]
 [ 1  0  0 17]
 [ 0  4  4  0]]
[[ 0  0  2  5]
 [ 0  0  0 11]
 [ 0  0  0  2]
 [ 0  0  0  1]
 [ 8  0  0 17]
 [ 0  8  7  0]]


Result: UEFZCUCJ